# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>

#### <center> **Final Project: ML** </center>
---

**Date**: 24 November, 2025

**Student Name**: Antonia Horburger

**Professor**: Pablo Camarillo Ramirez

## 1. Algoritmo de Machine Learning a usar

En este proyecto quiero construir un sistema de recomendación de películas.  
El objetivo es predecir qué tan bien le podría gustar una película a un usuario, a partir de las calificaciones que otros usuarios han dado en el pasado. Este problema es un problema típico de **sistemas de recomendación**, donde los datos principales son (usuario, ítem, rating).

Por eso elegí el algoritmo **Alternating Least Squares** de Spark MLlib. ALS es un algoritmo de factorización de matrices que funciona muy bien cuando tenemos muchos usuarios, muchos ítems y una matriz de calificaciones muy dispersa (la mayoría de las películas no han sido calificadas por la mayoría de los usuarios). Además, ALS está implementado de forma nativa en Apache Spark, lo que facilita el entrenamiento distribuido y el manejo de conjuntos de datos grandes.

## 2. Descripción del conjunto de datos

Para este proyecto utilizo el conjunto de datos **MovieLens "ml-latest-small"**.  
Este dataset se puede descargar desde la página de MovieLens / GroupLens o desde Kaggle (https://www.kaggle.com/datasets/shubhammehta21/movie-lens-small-latest-dataset). Contiene calificaciones de películas hechas por usuarios reales.

El archivo principal que uso es `ratings.csv`, que tiene las columnas `userId`, `movieId`, `rating` y `timestamp`.  
En total el dataset tiene alrededor de 100,000 calificaciones, más de 600 usuarios y casi 10,000 películas diferentes.  

Como se trata de un sistema de recomendación, es importante describir cuántos **usuarios únicos** y cuántos **ítems (películas) únicos** hay en el dataset. Esta descripción la hago usando PySpark, contando los usuarios distintos y las películas distintas en el DataFrame de calificaciones.


## 3. Proceso de entrenamiento del modelo de ML

El flujo de entrenamiento de mi sistema de recomendación es el siguiente:

1. Cargar el archivo ratings.csv en un DataFrame de Spark.
2. Dividir los datos en un conjunto de entrenamiento y un conjunto de prueba usando randomSplit.
3. Configurar el modelo **ALS** indicando qué columnas representan al usuario, al ítem y al rating.
4. Definir los hiperparámetros principales del modelo.
5. Entrenar el modelo usando el conjunto de entrenamiento.
6. Guardar el modelo entrenado en disco para poder cargarlo más tarde.

En este proyecto uso los siguientes hiperparámetros de ALS:

- rank: número de factores latentes. Controla la dimensión de las representaciones de usuario e ítem. Un valor moderado (por ejemplo, 10) permite capturar patrones sin hacer el modelo demasiado grande.
- maxIter: número máximo de iteraciones del algoritmo ALS. Más iteraciones permiten que el modelo converja mejor, pero aumentan el tiempo de cómputo. En este proyecto uso un valor como 10.
- regParam: parámetro de regularización. Sirve para evitar sobreajuste penalizando valores muy grandes en los vectores de usuario e ítem. Uso un valor como 0.1 para tener un equilibrio entre ajuste y generalización.
- nonnegative: si es True, fuerza a que los factores latentes sean no negativos. Esto puede hacer que las representaciones sean más interpretables.
- implicitPrefs: en este proyecto lo dejo en False porque mis ratings son explícitos (valores de 0.5 a 5.0 y no sólo información implícita de clics o vistas).
- coldStartStrategy: uso el valor "drop" para que Spark elimine las filas donde la predicción es NaN (por ejemplo, usuarios o películas no vistos en entrenamiento) antes de calcular las métricas.

Después de entrenar el modelo, lo guardo en una ruta dentro del sistema de archivos del cluster usando el método .save() de Spark ML.


In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Final Project - ALS Recommender") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

spark.conf.set("spark.sql.shuffle.partitions", "5")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/24 13:46:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql import functions as F


base_path = "/opt/spark/work-dir/data/ml/movies/"
ratings_path = base_path + "ratings.csv"

ratings_df = (spark.read
              .option("header", True)
              .option("inferSchema", True)
              .csv(ratings_path))

print("Schema:")
ratings_df.printSchema()

print("First 5 rows:")
ratings_df.show(5)

print("Total number of ratings:")
print(ratings_df.count())

print("Number of distinct users and movies:")
ratings_df.select(
    F.countDistinct("userId").alias("n_users"),
    F.countDistinct("movieId").alias("n_movies")
).show()


Schema:
root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

First 5 rows:
+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows
Total number of ratings:
100836
Number of distinct users and movies:


[Stage 6:>                                                          (0 + 1) / 1]

+-------+--------+
|n_users|n_movies|
+-------+--------+
|    610|    9724|
+-------+--------+



In [3]:
train_df, test_df = ratings_df.randomSplit([0.8, 0.2], seed=42)

print("Training set size:", train_df.count())
print("Test set size:", test_df.count())

Training set size: 80578
Test set size: 20258


In [4]:
from pyspark.ml.recommendation import ALS

als = ALS(
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    rank=10,             
    maxIter=10,           
    regParam=0.1,        
    nonnegative=True,    
    implicitPrefs=False,  
    coldStartStrategy="drop"  
)


als_model = als.fit(train_df)

## 4. Evaluación del modelo de ML

Para evaluar la calidad de mi sistema de recomendación con ALS utilizo la métrica **RMSE (Root Mean Squared Error)** sobre el conjunto de prueba.

El RMSE mide la diferencia promedio entre las calificaciones reales y las calificaciones predichas por el modelo. Cuanto más pequeño es el RMSE, mejor está ajustado el modelo a los datos. Esta métrica es adecuada porque en este problema los ratings son valores numéricos continuos (por ejemplo, de 0.5 a 5.0) y nos interesa que las predicciones estén lo más cerca posible de los valores reales.

El proceso de evaluación es:

1. Usar el modelo entrenado para generar predicciones sobre el conjunto de prueba con el método .transform(test_df).
2. Eliminar filas con predicciones NaN (esto ya lo hace automáticamente la opción coldStartStrategy="drop").
3. Calcular el RMSE usando RegressionEvaluator de Spark ML, indicando como etiqueta real la columna rating y como predicción la columna prediction.

Además, para entender mejor el comportamiento del modelo, también muestro algunas filas de las predicciones y algunas recomendaciones para usuarios específicos.

In [5]:
from pyspark.ml.evaluation import RegressionEvaluator

predictions = als_model.transform(test_df)

print("Predictions example:")
predictions.show(10, truncate=False)

evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

rmse = evaluator.evaluate(predictions)
print(f"Root-mean-square error (RMSE) = {rmse}")

Predictions example:


+------+-------+------+----------+----------+
|userId|movieId|rating|timestamp |prediction|
+------+-------+------+----------+----------+
|22    |858    |3.0   |1268726337|2.8674862 |
|22    |1193   |5.0   |1268726374|3.4953542 |
|22    |3006   |4.0   |1268726047|2.5993364 |
|22    |3481   |0.5   |1268726778|2.3403294 |
|22    |3489   |5.0   |1268726106|1.324918  |
|22    |4017   |4.0   |1268727100|2.3640177 |
|22    |4903   |4.0   |1268727442|1.5251408 |
|22    |5464   |2.0   |1268727450|2.835936  |
|22    |5669   |4.5   |1268726814|1.9405422 |
|22    |6874   |4.0   |1268726782|2.9470587 |
+------+-------+------+----------+----------+
only showing top 10 rows


Root-mean-square error (RMSE) = 0.877014014393235


## 4. Evaluación del modelo de ML

Para evaluar el desempeño de mi modelo ALS utilicé el conjunto de prueba, es decir, los datos que no fueron usados durante el entrenamiento. Primero generé predicciones para cada par (usuario, película) usando el método transform() y verifiqué que Spark eliminara automáticamente las filas con `NaN` gracias a la opción coldStartStrategy="drop".

Un ejemplo de las predicciones obtenidas es el siguiente:

- Usuario 22, Película 858 → rating real = 3.0, predicción ≈ 2.87  
- Usuario 22, Película 1193 → rating real = 5.0, predicción ≈ 3.49  
- Usuario 22, Película 3481 → rating real = 0.5, predicción ≈ 2.34  
- Usuario 22, Película 5669 → rating real = 4.5, predicción ≈ 1.94  

Estas diferencias entre la calificación real y la predicha son esperables en modelos de recomendación, ya que ALS intenta aproximar la matriz de usuarios y películas usando factores latentes.

Para medir la calidad del modelo utilicé la métrica **RMSE (Root Mean Squared Error)**.  
El valor obtenido fue:

**RMSE = 0.877**

Este valor indica que, en promedio, la diferencia entre la predicción del modelo y la calificación real es menor a 1 punto de rating (en una escala de 0.5 a 5). Este nivel de error es consistente con los resultados típicos de ALS en el dataset MovieLens “ml-latest-small”, y muestra que el modelo logra capturar patrones relevantes en las preferencias de los usuarios.